In [4]:
%%capture
import sys
import numpy as np

%pip install pyomo >/dev/null 2>/dev/null
%pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

import pandas as pd

In [6]:
model = pyo.ConcreteModel("National Park Trip")

# Sets
activity_table = pd.read_excel("data/activities.xlsx", sheet_name="Activities") # use correct path
activity_table = activity_table.set_index("Activity")
model.P = Set(initialize=['hikin', 'adventure', 'shopping', 'dining', 'vistas'])
model.K = Set(initialize=[1,2,3])
model.I = Set() #set of activities from the file
model.L = Set() #set of lodging options from its own file
model.N = Set() #set of all nodes in network flow (lodging + activities)

#Variables
model.x = pyo.Var(model.I, model.P, model.K, within=pyo.Binary) #activity selection 
model.y = pyo.Var(model.N, model.N, within=pyo.Binary) #path between locations (determines order)
model.h = pyo.Var(model.L, model.K, within=pyo.Binary) #lodging selection

#Objective Function
def total_cost(model):
    lodging_cost = sum(model.h[l, k] * activity_table.loc[l, 'Cost'] for l in model.L for k in model.K)
    activity_cost = sum(model.x[i,p,k] * actvity_table.loc[i, 'Cost'] for i in model.I for p in model.P for k in model.K)
    #car_cost = sum(model.y[i,j] * d_ij[i,j] for i in model.N for j in model.N) # need to re-do this constraint to reference the proper distance param from data tables
    return lodging_cost + activity_cost + car_cost

model.obj = pyo.Objective(rule=total_cost, sense=pyo.minimize)

#Constraints 
def scheduling_continutity(model, i):
    return sum(model.y[i,j] for j in model.N) == sum(model.x[i,p,k] for p in model.P for k in model.K)
model.scheduling_continuity = pyo.Constraint(model.I, rule=scheduling_continutity)

def hiking_time(model, k):
    return sum(model.x[i, 'hiking', k] * activity_table.loc[i, 'Time'] for i in model.I) <= 5
model.hiking_time = pyo.Constraint(model.K, rule=hiking_time)

def adventure_budget(model):
    return sum(model.x[i, 'adventure', k] * activity_table.loc[i, 'Cost'] for i in model.I) <= 600
model.adventure_budget = pyo.Constraint(model.K, rule=adventure_budget)

def travel_time(model):
    return sum(model.y[i,j] * r_ij[i,j] for i,j in model.N) <= sum(model.x[i,p,k] * t_ip[i] for i in model.I for p in model.P for k in model.K)
model.travel_time = pyo.Constraint(model.K, rule=travel_time)

def min_fun(model):
    return sum(model.x[i,p,k] * activity_table.loc[i, 'Fun'] for i in model.I for p in model.P for k in model.K) >= 100
model.min_fun = pyo.Constraint(model.K, rule=min_fun)   




FileNotFoundError: [Errno 2] No such file or directory: 'data/activities.xlsx'